In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from concurrent.futures import ThreadPoolExecutor

## Read video file and applay mask

In [16]:
#load mp4 test video
cap = cv2.VideoCapture('test_data/test_pallina_piccola_telefono.mp4')

start_time = 11.8 #set video start position (in seconds)
end_time = 18 #set video end position (in seconds)

#set video start position (in milliseconds)
cap.set(cv2.CAP_PROP_POS_MSEC, start_time*1000)

#get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

#create a mask to get only the tube
ret, frame = cap.read()
mask = np.zeros_like(frame)
cv2.rectangle(mask, (150, 50), (380, 800), (255, 255, 255), -1)


#apply mask to video and put frames in a list
last_frame = int(end_time*fps)
frame_counter = 0
frames = []
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == True:
        frame = cv2.bitwise_and(frame, mask)
        frames.append(frame)
        frame_counter += 1
    else:
        break

cap.release()


In [21]:
print(fps)

29


In [20]:
#write video with mask applied
mask_width = 380 - 150
mask_height = 800 - 50
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('test_data/ret_masked.avi', fourcc, fps, (mask_width, mask_height))
for i, frame in enumerate(frames):
    if i * (1/fps) > 5.5:
        break
    cropped_frame = frame[50:800, 150:380]
    out.write(cropped_frame)
out.release()


## Compute difference between frames $n$ and $n-1$
we compute the difference and then produce a video to see the result

In [4]:
#compute difference between consecutive frames
diff_frames = []
for i in range(len(frames)-1):
    diff = cv2.absdiff(frames[i], frames[i+1])
    diff_frames.append(diff)

In [5]:
#create video with difference frames
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('test_data/diff.avi', fourcc, fps, (frame_width, frame_height))
for i in range(len(diff_frames)):
    out.write(diff_frames[i])
out.release()


## Create new mask
The mask is made in the following wat:
1) find centroids of "light" ball using KMeans
2) make circular mask at centroids

In [6]:
import warnings

warnings.filterwarnings("ignore", message="KMeans is known to have a memory leak on Windows")

In [7]:
bw_frames = []
for i in range(len(diff_frames)):
    bw = cv2.cvtColor(diff_frames[i], cv2.COLOR_BGR2GRAY)
    #rescale image to 0-255
    bw = cv2.normalize(bw, None, 0, 255, cv2.NORM_MINMAX)
    bw_frames.append(bw)


In [8]:
#find centroid of the ball in each frame using k-means clustering (k=1)
def find_centroid(frame):
    y, x = np.where(frame == 255)
    coordinates = np.column_stack([x, y])
    if len(coordinates) > 0:
        kmeans = KMeans(n_clusters=1)
        kmeans.fit(coordinates)
        centroid = kmeans.cluster_centers_
        cv2.circle(frame, (int(centroid[0][0]), int(centroid[0][1])), 10, (0, 0, 255), 5)
        return centroid

In [9]:
with ThreadPoolExecutor() as executor:
    centroids = list(executor.map(find_centroid, bw_frames))


In [10]:
#create a new mask to better isolate the falling ball
#mask is circular and is centered in the centroid of the ball

#last frame uses same centroid as previous frame
centroids.append(centroids[-1])

frames_msked = []
for i in range(len(frames)):
    mask = np.zeros_like(frames[0])
    cv2.circle(mask, (int(centroids[i][0][0]), int(centroids[i][0][1])), 20, (255, 255, 255), -1)
    frame = cv2.bitwise_and(frames[i], mask)
    frames_msked.append(frame)


In [11]:
#make a video with masked frames
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('test_data/masked.avi', fourcc, fps, (frame_width, frame_height))
for i in range(len(frames_msked)):
    out.write(frames_msked[i])
out.release()

## Track the ball

In [12]:
def getLimits(b,g,r,rangeWidth):
    arr = [[[b,g,r]]]
    color = np.uint8(arr)
    hsvColor = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)
    lowerLimit = [hsvColor[0][0][0]-rangeWidth,40,40]
    upperLimit = [hsvColor[0][0][0]+rangeWidth,255,255]
    return [lowerLimit,upperLimit]

In [15]:
def frameCenter(frame,color,width,area):
    b,g,r = color
    lims=getLimits(b,g,r,width)
    lowerLimit = lims[0]
    upperLimit = lims[1]
    hsv_image =  cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    blue_regions = cv2.inRange(hsv_image, np.array(lowerLimit), np.array(upperLimit))#lower and upper limit are to be enclosed in an np array
    
    contours, _ = cv2.findContours(blue_regions, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)#CHIEDI
#     blue_dots_positions = []

    for contour in contours:
#       print(f"area = {cv2.contourArea(contour)}")
      if cv2.contourArea(contour) > area: # Minimum contour area threshold
        M = cv2.moments(contour)
        center_x = M["m10"] / M["m00"]
        center_y = M["m01"] / M["m00"]#CHIEDI
        print(f"Centroid of dot: (x, y) = ({center_x:.1f}, {center_y:.1f})")
#         plt.scatter(center_x, center_y, color='blue', marker='o', facecolor='none', s=200);#s is the size of the scattered contour
#         blue_dots_positions.append([center_x, center_y])
        return (center_x,center_y)

In [18]:
#track the ball in each frame
color = (52,58,62)
width = 18
area = 100

frame_centers = []
for i in range(len(frames_msked)):
    frame = frames_msked[i]
    frame_centers.append(frameCenter(frame,color,width,area))

print(frame_centers)

#unpack x,y coordinates tuples
x = [x[0] for x in frame_centers]
y = [x[1] for x in frame_centers]

#plot ball trajectory
plt.figure(figsize=(10,10))
plt.plot([x[0] for x in frame_centers], [x[1] for x in frame_centers], color='blue', marker='o', linestyle='dashed', linewidth=2, markersize=12)
plt.xlim(0, frame_width)
plt.ylim(frame_height, 0)
plt.show()
    

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, Non

TypeError: 'NoneType' object is not subscriptable